# Try 13: setup `paprika` restraints

Presuming we have a host-guest system already setup, let's see if we can rationally instruct `paprika` to setup all the restraints for a real system.

In [1]:
%load_ext autoreload
%autoreload 2

import os as os
import glob as glob
import numpy as np
import pandas as pd

import parmed as pmd

We are going to setup host translational restraints, guest translational restraints, host "jacks," and guest wall restraints with `paprika`. First, let's try working in a directory where we have a known `disang.rest` file we can use as a reference: `/data/davids4/projects/smirnoff-host-guest/a-bam-p/a00/original` or `/data/davids4/projects/smirnoff-host-guest/a-bam-p/u00/original` for the final values.

Since we're starting with files Niel prepared, the dummy atoms are first, followed by the host, and then the guest atoms.

In [30]:
dummy_anchors = [':1', ':2', ':3']
host_anchors  = [':4@O3', ':6@C1', ':8@C6']
guest_anchors = [':10@C4', ':10@N1']

code_mapping = {
    'D0' : dummy_anchors[0],
    'D1' : dummy_anchors[1],
    'D2' : dummy_anchors[2],
    'H0' : host_anchors[0],
    'H1' : host_anchors[1],
    'H2' : host_anchors[2],
    'G0' : guest_anchors[0],
    'G1' : guest_anchors[1]
}


The attach fractions can be found in the `Setup.pl` file in the original location on `kirkwood`: `/data/nhenriksen/projects/cds/wat6/bgbg-tip3p/a-bam-p/a00/` under `@AttachFC`.

In [3]:
attach_string = '0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 18.10 24.40 37.00 49.60 74.80'
attach_fractions = [float(i) / 100 for i in attach_string.split()]

The attach force constant is 5.0 kcal/mol-A^{2} for distances and 100.0 kcal/mol-rad^{2} for angels and torsions based on `disang.rest` in `u00`.

In [4]:
attach_distance_fc = 5.0
attach_angle_fc    = 100.0

The pull distances are specified by `@Translate` in `Setup.pl` and are offset by 6 Angstroms.

In [5]:
pull_string = '0.00 0.40 0.80 1.20 1.60 2.00 2.40 2.80 3.20 3.60 4.00 4.40 4.80 5.20 5.60 6.00 6.40 6.80 7.20 7.60 8.00 8.40 8.80 9.20 9.60 10.00 10.40 10.80 11.20 11.60 12.00 12.40 12.80 13.20 13.60 14.00 14.40 14.80 15.20 15.60 16.00 16.40 16.80 17.20 17.60 18.00'
pull_distances = [float(i) + 6.00 for i in pull_string.split()]

## Load in the structure

In [6]:
hg = pmd.load_file('systems/a-bam-p/a00/original/full.hmr.topo',
                   'systems/a-bam-p/a00/original/full.crds',
                    structure=True)

In [12]:
windows = [len(attach_fractions), len(pull_distances), 0]
print(f'There are {windows} windows in this attach-pull calculation.')

DEBUG:root:OpenMM support: Yes


There are [14, 46, 0] windows in this attach-pull calculation.


## Initialize `paprika` restraints

First, let's add "static" restraints where the value of the restraint doesn't change over the course of the simulation.

In [33]:
import paprika
print(os.path.abspath(paprika.__file__))
from paprika.restraints import static_DAT_restraint
from paprika.restraints import DAT_restraint

/home/dslochower/hgst-3tb-data/projects/pAPRika/paprika/__init__.py


In [14]:
host_restraints = []

The translational distance of the host...

In [20]:
# Translational distance...
# Anchored with last dummy atom - first host atom
static_DAT_restraint([dummy_anchors[2], host_anchors[0]], 
                     windows, hg, 
                     attach_distance_fc, amber_index=True)

# Translational angle...
# Anchored with first two dummy atoms - first host atom
static_DAT_restraint([dummy_anchors[0], dummy_anchors[1], host_anchors[0]], 
                     windows, hg,
                     attach_angle_fc, amber_index=True)

# Translational dihedral...
# Anchored with three dummy atoms - first host atom
static_DAT_restraint([dummy_anchors[0], dummy_anchors[1], dummy_anchors[2], host_anchors[0],], 
                     windows, hg, attach_angle_fc, amber_index=True)

# Orientational angle
# Anchored with the last dummy atom - first two host atoms...
static_DAT_restraint([dummy_anchors[2], host_anchors[0], host_anchors[1]], 
                     windows, hg, attach_angle_fc, amber_index=True)

# Orientational dihedral 1
# Anchored with the last two dummy atoms - first two host atoms...
static_DAT_restraint([dummy_anchors[0], dummy_anchors[1], host_anchors[0], host_anchors[1]], 
                     windows, hg, attach_angle_fc, amber_index=True)

# Orientational dihedral 2
# Anchored with the last dummy atom - first three host atoms..
static_DAT_restraint([dummy_anchors[1], host_anchors[0], host_anchors[1], host_anchors[2]], 
                     windows, hg, attach_angle_fc, amber_index=True)

DEBUG:root:Calculating attach targets and force constants...
DEBUG:root:Attach, Method #1
DEBUG:root:Calculating pull targets and force constants...
DEBUG:root:Pull, Method #1
DEBUG:root:Calculating release targets and force constants...
DEBUG:root:No restraint info set for the release phase! Skipping...
DEBUG:root:Number of attach windows = 14
DEBUG:root:Number of pull windows = 46
DEBUG:root:This restraint will be skipped in the release phase
DEBUG:root:Assigning atom indices...
DEBUG:root:There are 1 atoms in the mask :3  ...
DEBUG:root:There are 1 atoms in the mask :4@O3  ...
DEBUG:root:Calculating attach targets and force constants...
DEBUG:root:Attach, Method #1
DEBUG:root:Calculating pull targets and force constants...
DEBUG:root:Pull, Method #1
DEBUG:root:Calculating release targets and force constants...
DEBUG:root:No restraint info set for the release phase! Skipping...
DEBUG:root:Number of attach windows = 14
DEBUG:root:Number of pull windows = 46
DEBUG:root:This restraint w

In [29]:
static_restraints = [['D2', 'H0'],
                     ['D0', 'D1', 'H0'],
                     ['D0', 'D1', 'D2', 'H0'],
                     ['D2', 'H0', 'H1'],
                     ['D0', 'D1', 'H0', 'H1'],
                     ['D1', 'H0', 'H1', 'H2'],
                    ]

static_restraint_list = []
for static_restraint in static_restraints:
    static_restraint_list.append(
        static_DAT_restraint([code_mapping[mask] for mask in static_restraint], 
                             windows, hg,
                             attach_angle_fc, amber_index=True))

DEBUG:root:Calculating attach targets and force constants...
DEBUG:root:Attach, Method #1
DEBUG:root:Calculating pull targets and force constants...
DEBUG:root:Pull, Method #1
DEBUG:root:Calculating release targets and force constants...
DEBUG:root:No restraint info set for the release phase! Skipping...
DEBUG:root:Number of attach windows = 14
DEBUG:root:Number of pull windows = 46
DEBUG:root:This restraint will be skipped in the release phase
DEBUG:root:Assigning atom indices...
DEBUG:root:There are 1 atoms in the mask :3  ...
DEBUG:root:There are 1 atoms in the mask :4@O3  ...
DEBUG:root:Calculating attach targets and force constants...
DEBUG:root:Attach, Method #1
DEBUG:root:Calculating pull targets and force constants...
DEBUG:root:Pull, Method #1
DEBUG:root:Calculating release targets and force constants...
DEBUG:root:No restraint info set for the release phase! Skipping...
DEBUG:root:Number of attach windows = 14
DEBUG:root:Number of pull windows = 46
DEBUG:root:This restraint w

Let's define the guest translational distance, translational angle, and orientational angle relative to the dummy atoms.

In [36]:
guest_restraints = [
    ['D2', 'G0'],
    ['D1', 'D2', 'G0'],
    ['D2', 'G0', 'G1']
]

guest_targets = [
    [6.0],
    [180.0],
    [180.0]
]

In [42]:
for restraint in guest_restraints:
    if len(restraint) > 2:
        angle = True
    else:
        angle = False
    this = DAT_restraint()
    this.auto_apr = True
    this.amber_index = True
    this.topology = hg
    this.attach['fraction_list'] = attach_fractions
    this.pull['num_windows'] = len(pull_distances)
    this.mask1 = code_mapping[restraint[0]]
    this.mask2 = code_mapping[restraint[1]]
    
    if angle:
        this.mask3 = code_mapping[restraint[2]]
        this.attach['target'] = 180.0
        this.pull['target_final'] = 180.0
        this.attach['fc_final'] = attach_angle_fc
    else:
        this.attach['target'] = pull_distances[0]
        this.pull['target_final'] = pull_distances[-1]
        this.attach['fc_final'] = attach_distance_fc
    
    this.initialize()

DEBUG:root:Calculating attach targets and force constants...
DEBUG:root:Attach, Method #3
DEBUG:root:Calculating pull targets and force constants...
DEBUG:root:Pull, Method #1
DEBUG:root:Calculating release targets and force constants...
DEBUG:root:No restraint info set for the release phase! Skipping...
DEBUG:root:Number of attach windows = 14
DEBUG:root:Number of pull windows = 46
DEBUG:root:This restraint will be skipped in the release phase
DEBUG:root:Assigning atom indices...
DEBUG:root:There are 1 atoms in the mask :3  ...
DEBUG:root:There are 1 atoms in the mask :10@C4  ...
DEBUG:root:Calculating attach targets and force constants...
DEBUG:root:Attach, Method #3
DEBUG:root:Calculating pull targets and force constants...
DEBUG:root:Pull, Method #1
DEBUG:root:Calculating release targets and force constants...
DEBUG:root:No restraint info set for the release phase! Skipping...
DEBUG:root:Number of attach windows = 14
DEBUG:root:Number of pull windows = 46
DEBUG:root:This restraint 

Great, all we need now are the jacks, and the wall restraints!